In [1]:
%matplotlib inline
import datetime
import json

import iso8601
import pandas as pd

import modules.pd_utils as pd_utils
import modules.gtimeline_wrapper as gtwrapper

In [2]:
csv_path = 'derived_data/02_man_marked_moved_frames/manuel_Cam_0.csv'

In [3]:
df = pd.read_csv(csv_path, sep=';')

In [4]:
df_cam_ts_fst = df.apply(lambda row: pd_utils.extract_metadata_videoname(row['first_frame']), axis = 1)
df_cam_ts_sec = df.apply(lambda row: pd_utils.extract_metadata_videoname(row['second_frame']), axis = 1)

df_cam_ts_fst.columns = ['fst_cam_id', 'fst_ts_start', 'fst_ts_end']
df_cam_ts_sec.columns = ['sec_cam_id', 'sec_ts_start', 'sec_ts_end']

df = df.merge(df_cam_ts_fst, left_index=True, right_index=True)
df = df.merge(df_cam_ts_sec, left_index=True, right_index=True)

Die Frames stammen immer aus dem ersten Frame eines Videos, welches um die Uhrzeit 0 aufgenommen wurde. Eine Video wie

`Cam_0_2016-07-20T23:55:06.659237Z--2016-07-21T00:00:54.807835Z.jpg`

wird als Stellvertreter für den Tag `2016-07-21` betrachtet.

In [5]:
def get_proxy_day_start(row, column):
    """Retruns the day of the timestamp in `column`"""
    return iso8601.parse_date(row[column].strftime("%Y-%m-%d"))

def get_proxy_day_end(row, column):
    """Retruns the day of the timestamp in `column`"""
    return iso8601.parse_date(row[column].strftime("%Y-%m-%d")) + datetime.timedelta(0, 86340)

In [6]:
df['proxy_day_start'] = df.apply(lambda row: get_proxy_day_start(row, 'fst_ts_end'), axis = 1)
df['proxy_day_end'] = df.apply(lambda row: get_proxy_day_end(row, 'fst_ts_end'), axis = 1 )

In [7]:
df[:1]

,first_frame,second_frame,moved,fst_cam_id,fst_ts_start,fst_ts_end,sec_cam_id,sec_ts_start,sec_ts_end,proxy_day_start,proxy_day_end
0,Cam_0_2016-07-20T23:55:06.659237Z--2016-07-21T...,Cam_0_2016-07-21T23:57:12.754348Z--2016-07-22T...,1,0,2016-07-20 23:55:06.659237+00:00,2016-07-21 00:00:54.807835+00:00,0,2016-07-21 23:57:12.754348+00:00,2016-07-22 00:02:52.599861+00:00,2016-07-21 00:00:00+00:00,2016-07-21 23:59:00+00:00


# Vorbereitung für das Darstellen mittels gTimeline

In [8]:
def add_tooltip(row):
    string="""
    <table style="font-size:10px">
        <tr>
            <td><b>Moved:</b></td><td>{moved}</td>
        </tr>
        <tr>
            <td><b>Day:</b></td> <td>{day}</td>
        </tr>
        <tr>
            <td><b>1. Reference Image:</b></td> <td>{fst_frame}</td>
        </tr>
        <tr>
            <td><b>2. Reference Image:</b></td> <td>{sec_frame}</td>
        </tr>
    </table>
    """.format(moved=row['moved'],
               day=row['proxy_day_start'].strftime('%Y-%m-%d'),
               fst_frame=row['first_frame'],
               sec_frame=row['second_frame'])
    return string

In [9]:
df['tooltip'] = df.apply(add_tooltip, axis = 1)
df['row_label'] = 'Cam_0 manual marked'

In [10]:
export_df = df.copy()
export_df = export_df[['row_label','proxy_day_start', 'proxy_day_end','moved', 'tooltip']]
export_df.columns=['row_label', 'start', 'end', 'bar_label', 'tooltip']

In [11]:
export_df.to_pickle('./derived_data/02_man_marked_moved_frames/df_man_marked.pkl')

In [12]:
g_dict = gtwrapper.create_gtimeline_dict(export_df, 'row_label', 'bar_label', 'start', 'end', 'tooltip')

with open('docs/02_mark_moved_frames_manual/data.json', 'w') as fp:
    json.dump(g_dict, fp)

In [13]:
!firefox -p develop docs/02_mark_moved_frames_manual/02_mark_moved_frames_manual.html